In [ ]:
!wget -O OD_SS.zip https://arquivos.ufsc.br/f/981acdaa6d2442f68c1a/?dl=1

In [ ]:
!unzip OD_SS.zip

In [ ]:
!mkdir FeulgenDataset
!mkdir FeulgenDataset/train
!mkdir FeulgenDataset/train/Images
!mkdir FeulgenDataset/train/Labels
!mkdir FeulgenDataset/test
!mkdir FeulgenDataset/test/Images
!mkdir FeulgenDataset/test/Labels
!mkdir FeulgenDataset/val
!mkdir FeulgenDataset/val/Images
!mkdir FeulgenDataset/val/Labels

In [ ]:
!cp -r /content/OD_SS/train/Images/* /content/FeulgenDataset/train/Images
!cp -r /content/OD_SS/test/Images/* /content/FeulgenDataset/test/Images
!cp -r /content/OD_SS/val/Images/* /content/FeulgenDataset/val/Images
!cp -r /content/OD_SS/train/OD_Labels/* /content/FeulgenDataset/train/Labels
!cp -r /content/OD_SS/test/OD_Labels/* /content/FeulgenDataset/test/Labels
!cp -r /content/OD_SS/val/OD_Labels/* /content/FeulgenDataset/val/Labels

In [ ]:
!pip install pyyaml==5.1
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
import os
import cv2
import json
import io
import base64
import fnmatch
import itertools
import PIL
import csv
import sys
import numpy as np

import detectron2
from detectron2.utils.logger import setup_logger

from detectron2.data import DatasetCatalog, MetadataCatalog

from detectron2.structures import BoxMode

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer

In [ ]:
def get_data_dicts(base_path, folder):

    img_dir = os.path.join(base_path, folder, "Images")
    labels_dir = os.path.join(base_path, folder, "Labels")

    list_of_files = os.listdir(img_dir)
    dataset_dicts = []

    for idx, item in enumerate(list_of_files):
        height, width = cv2.imread(os.path.join(img_dir, item)).shape[:2]

        record = {}
        record["file_name"] = os.path.join(img_dir, item)
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        objs = []

        with open(os.path.join(labels_dir, item.replace("png", "txt"))) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')

            for bbox in csv_reader:
                (startX, startY, endX, endY, label) = int(bbox[0].strip()), int(bbox[1].strip()), int(bbox[2].strip()), int(bbox[3].strip()), bbox[4].strip()

                px = [startX, endX]
                py = [startY, endY]
                poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
                poly = [p for x in poly for p in x]

                if label == "alterada":
                    category_id = 0
                elif label == "intermediaria":
                    category_id = 1
                elif label == "velhas":
                    category_id = 2
                elif label == "sujeira":
                    category_id = 3
                elif label == "sobreposicao":
                    category_id = 4
                elif label == "nao_identificado":
                    category_id = 5
                elif label == "neutrofilo":
                    category_id = 6
                elif label == "mancha":
                    category_id = 7
                else:
                    print("Erro: classe inválida!")
                    sys.exit(0)

                obj = {
                    "bbox": [startX, startY, endX, endY],
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "segmentation": [poly],
                    "category_id": category_id,
                }
                objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts

In [ ]:
MODEL = "faster_rcnn_R_101_FPN_3x"
root = "/content/FeulgenDataset"

for d in ["train", "val"]:
    DatasetCatalog.register(os.path.join(root, d), lambda d=d: get_data_dicts(root, d))
    MetadataCatalog.get(os.path.join(root, d)).set(
    thing_classes=['alterada', 'intermediaria', 'velhas', 'sujeira', 'sobreposicao', 'nao_identificado', 'neutrofilo', 'mancha'])

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/" + MODEL + ".yaml"))
cfg.DATASETS.TRAIN = (os.path.join(root, "train"),)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/" + MODEL + ".yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 15000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.RETINANET.NUM_CLASSES = 8

os.makedirs(os.path.join("Output", MODEL), exist_ok=True)
cfg.OUTPUT_DIR = os.path.join("Output", MODEL)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()